In [1]:
import keras
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend

from tensorflow.keras.layers import Dense, Dropout, Activation, LSTM, GRU
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.losses import cosine_similarity,mse

import csv
import sklearn
#import keras.backend.tensorflow_backend as K

import art

#from tensorflow.compat.v1.keras.backend import get_session

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import time as time
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import classification_report

%matplotlib inline

pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

#pip install adversarial-robustness-toolbox >/dev/null
from art.attacks.evasion import FastGradientMethod, BasicIterativeMethod, DeepFool, SaliencyMapMethod, CarliniL2Method, CarliniLInfMethod
from art.classifiers import PyTorchClassifier
from art.estimators.classification import KerasClassifier
from art.defences.detector.poison import PoisonFilteringDefence

Using TensorFlow backend.


### Data Loading

In [2]:
data = []
X = []
Y = []

data1 = []
X1 = []
Y1 = []

data2 = []
X2 = []
Y2 = []

In [3]:
with open('D:/Data/OctFile.txt', newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=',')
    for row in csvreader:
        data.append(row)
        #print(row)
seed = 10
test_size = 0.2
Train_data,Test_data= train_test_split(data, test_size= test_size, random_state=seed)
print("***********************Load dataset 1 ******************************************************")


with open('D:/Data/NovFile.txt', newline='') as csvfile1:
    csvreader1 = csv.reader(csvfile1, delimiter=',')
    for row1 in csvreader1:
        data1.append(row1)
        #print(row)
seed = 10
test_size = 0.2
Train_data1,Test_data1= train_test_split(data1, test_size= test_size, random_state=seed)

print("***********************Load dataset 2 ******************************************************")


with open('D:/Data/DecFile.txt', newline='') as csvfile2:
    csvreader2 = csv.reader(csvfile2, delimiter=',')
    for row2 in csvreader2:
        data2.append(row2)
        #print(row)
seed = 10
test_size = 0.2
Train_data2,Test_data2= train_test_split(data2, test_size= test_size, random_state=seed)
print("***********************Load dataset 3 ******************************************************")

***********************Load dataset 1 ******************************************************
***********************Load dataset 2 ******************************************************
***********************Load dataset 3 ******************************************************


### Dividing Data to Train and Test

In [4]:
for i in data:
    X.append(i[:14])       
    Y.append(i[14:19])         
    
for j in data1:
    X1.append(j[:14])        
    Y1.append(j[14:19])         
    
for k in data2:
    X2.append(k[:14])        
    Y2.append(k[14:19])         

In [5]:
seed = 10
test_size = 0.2
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size= test_size, random_state=seed)
#test_size = 0.25
#X_train, X_val, Y_train, Y_val= train_test_split(X_train, Y_train, test_size= test_size, random_state=seed)
X_train1, X_test1, Y_train1, Y_test1= train_test_split(X1, Y1, test_size= test_size, random_state=seed)

X_train2, X_test2, Y_train2, Y_test2= train_test_split(X2, Y2, test_size= test_size, random_state=seed)

In [ ]:
print("X_train=", len(X_train))
print("Y_train=", len(Y_train))
print("X_test=",len(X_test)) 
print("Y_test=",len(Y_test)) 
print("*******************************************************")

print("X_train1=", len(X_train1))
print("Y_train1=", len(Y_train1))
print("X_test1=",len(X_test1)) 
print("Y_test1=",len(Y_test1))
print("*******************************************************")

print("X_train2=", len(X_train2))
print("Y_train2=", len(Y_train2))
print("X_test2=",len(X_test2)) 
print("Y_test2=",len(Y_test2))

In [7]:
from sklearn.preprocessing import Normalizer

import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import numpy as np

Norm = Normalizer().fit(X_train)
X_train = Norm.transform(X_train)

Norm = Normalizer().fit(Y_train)
Y_train = Norm.transform(Y_train)

Norm = Normalizer().fit(X_test)
X_test = Norm.transform(X_test)

Norm = Normalizer().fit(Y_test)
Y_test = Norm.transform(Y_test)

X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
Y_train = np.reshape(Y_train, (Y_train.shape[0], Y_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))
Y_test = np.reshape(Y_test, (Y_test.shape[0],  Y_test.shape[1]))

#****************************************************************************************

Norm1 = Normalizer().fit(X_train1)
X_train1 = Norm1.transform(X_train1)

Norm1 = Normalizer().fit(Y_train1)
Y_train1 = Norm1.transform(Y_train1)

Norm1 = Normalizer().fit(X_test1)
X_test1 = Norm1.transform(X_test1)

Norm1 = Normalizer().fit(Y_test1)
Y_test1 = Norm1.transform(Y_test1)

X_train1 = np.reshape(X_train1, (X_train1.shape[0], 1, X_train1.shape[1]))
Y_train1 = np.reshape(Y_train1, (Y_train1.shape[0], Y_train1.shape[1]))
X_test1 = np.reshape(X_test1, (X_test1.shape[0], 1, X_test1.shape[1]))
Y_test1 = np.reshape(Y_test1, (Y_test1.shape[0],  Y_test1.shape[1]))
#****************************************************************************************

Norm2 = Normalizer().fit(X_train2)
X_train2 = Norm2.transform(X_train2)

Norm2 = Normalizer().fit(Y_train2)
Y_train2 = Norm2.transform(Y_train2)

Norm2 = Normalizer().fit(X_test2)
X_test2 = Norm2.transform(X_test2)

Norm2 = Normalizer().fit(Y_test2)
Y_test2 = Norm2.transform(Y_test2)

X_train2 = np.reshape(X_train2, (X_train2.shape[0], 1, X_train2.shape[1]))
Y_train2 = np.reshape(Y_train2, (Y_train2.shape[0], Y_train2.shape[1]))
X_test2 = np.reshape(X_test2, (X_test2.shape[0], 1, X_test2.shape[1]))
Y_test2 = np.reshape(Y_test2, (Y_test2.shape[0],  Y_test2.shape[1]))


## In case you want to use each Model, un-comment it.

### Model 1

In [8]:
#model = Sequential()
#model.add(LSTM(256,input_shape=(None,14), return_sequences=True))  
#model.add(Dropout(0.1))
#model.add(LSTM(256,return_sequences=True))  
#model.add(Dropout(0.1))
#model.add(LSTM(256, return_sequences=True))  
#model.add(Dropout(0.1))
#model.add(LSTM(256, return_sequences=False))  
#model.add(Dropout(0.1))
#model.add(Dense(5,activation='softmax'))
#model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
#model.summary()

### Model 2

In [ ]:
#model = Sequential()
#model.add(LSTM(256, input_shape=(None,14)))
#model.add(Dropout(0.2))
#model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.2))
#model.add(Dense(5, activation='softmax'))

#opt = tf.keras.optimizers.Adam(learning_rate=0.005)
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.7, nesterov=True)
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2,decay_steps=10000,decay_rate=0.9)
#optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)
#model.compile(loss='mse',optimizer=opt,metrics=['accuracy'])

#model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

### Compile Model

In [ ]:
model1=model
model2=model
history = model.fit(X_train, Y_train,
                    epochs=200,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model.evaluate(X_test, Y_test, batch_size=14)
    
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy

print("*****************************************************************************")
history1 = model1.fit(X_train1, Y_train1,
                    epochs=200,
                    batch_size=14, validation_split=0.1, verbose=1)
score1 = model1.evaluate(X_test1, Y_test1, batch_size=14)
    
print("%s: %.2f%% :" % (model1.metrics_names[0], score1[0]*100)) #Loss
print("%s: %.2f%% :" % (model1.metrics_names[1], score1[1]*100)) #Accuracy


print("*****************************************************************************")
history2 = model2.fit(X_train2, Y_train2,
                    epochs=200,
                    batch_size=14, validation_split=0.1, verbose=1)
score2 = model2.evaluate(X_test2, Y_test2, batch_size=14)
    
print("%s: %.2f%% :" % (model2.metrics_names[0], score2[0]*100)) #Loss
print("%s: %.2f%% :" % (model2.metrics_names[1], score2[1]*100)) #Accuracy

In [11]:
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', 'cosine'])

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],"-b", label="Oct")
plt.plot(history1.history['accuracy'],"-r", label="Nov")
plt.plot(history2.history['accuracy'],"-g", label="Dec")
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc="best")
#plt.plot(history.history['accuracy'],"-b", label="Jan")
#plt.plot(history1.history['accuracy'],"-r", label="Feb")
#plt.plot(history2.history['accuracy'],"-g", label="Mar")
plt.show()

#plt.plot(history.history['cosine'])
#plt.ylabel('cosine')
#plt.xlabel('Epoch')
#plt.show()

In [10]:
from keras import backend
import tensorflow as tf
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
classifier = KerasClassifier(model=model2, use_logits=True)
fgsm = FastGradientMethod(classifier, 
                          eps=.01,
                          minimal=True,
                          eps_step=0.01,
                          num_random_init=35,
                          targeted=False,
                          batch_size=14)

x_train_adv_fgsm = fgsm.generate(x=X_train)
history = model1.fit(x_train_adv_fgsm, Y_train,
                    epochs=2,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model1.evaluate(X_test, Y_test, batch_size=14)
    
print("%s: %.2f%% :" % (model1.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model1.metrics_names[1], score[1]*100)) #Accuracy


In [ ]:
classifier2 = KerasClassifier(model=model2, use_logits=False)
sla=SaliencyMapMethod(classifier2,batch_size=32)
x_train_adv_sla=sla.generate(x=X_train)

history = model2.fit(x_train_adv_sla, Y_train,
                    epochs=20,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model2.evaluate(X_test, Y_test, batch_size=14)
    
print("%s: %.2f%% :" % (model2.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model2.metrics_names[1], score[1]*100)) #Accuracy

In [ ]:
from art.defences.transformer.evasion import DefensiveDistillation
from art.defences.preprocessor import AdversarialTraining
#,batch_size= 128, nb_epochs= 10
newclassifier=AdversarialTraining(X_train_adv,Y_train)
#X_corrected = defence1.generate(x=X_train_adv)
history = newclassifier.fit(X_test, Y_test,
                   batch_size=14,  verbose=1)
#score = classifier.evaluate(X_test, Y_test, batch_size=14)
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy
#print("%s: %.2f%% :" % (model.metrics_names[2], score[2]*100)) #  cosine

In [ ]:
attack2 = BasicIterativeMethod(estimator=classifier, eps= 0.3, eps_step= 0.1, max_iter= 1, targeted= False, batch_size= 32
                               ,verbose= False)
X_train_adv2 = attack2.generate(x=X_train)
history = model2.fit(X_train_adv, Y_train,
                    epochs=2,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model2.evaluate(X_test, Y_test, batch_size=14)
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy
#print("%s: %.2f%% :" % (model.metrics_names[2], score[2]*100)) #  cosine

In [ ]:
from art.defences.transformer.poisoning  import DefensiveDistillation

defence2 =DefensiveDistillation(classifier=classifier)
#X_corrected = defence2.generate(x=X_train_adv)
history = model2.fit(X_train_adv2, Y_train,
                    epochs=10,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model2.evaluate(X_test, Y_test, batch_size=14)
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy
#print("%s: %.2f%% :" % (model.metrics_names[2], score[2]*100)) #  cosine

In [ ]:
attack = DeepFool(classifier=classifier,max_iter= 5, epsilon= 1e-06, nb_grads= 10, batch_size= 1, verbose= False)
X_train_adv = attack.generate(x=X_train)
history = model.fit(X_train_adv, Y_train,
                    epochs=2,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model.evaluate(X_test, Y_test, batch_size=14)
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy
#print("%s: %.2f%% :" % (model.metrics_names[2], score[2]*100)) #  cosine

### Some other methods which we have not presented in the paper

In [ ]:
attack = SaliencyMapMethod(classifier=classifier, theta= 0.1, gamma= 1.0, batch_size= 1, verbose= False)
X_train_adv = attack.generate(x=X_train)
history = model.fit(X_train_adv, Y_train,
                    epochs=2,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model.evaluate(X_test, Y_test, batch_size=14)
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy
#print("%s: %.2f%% :" % (model.metrics_names[2], score[2]*100)) #  cosine

In [ ]:
attack = CarliniL2Method(classifier=classifier, confidence= 0.0, targeted= False, learning_rate= 0.01, binary_search_steps= 10
                         , max_iter= 1, initial_const= 0.01, max_halving= 5, max_doubling= 5, batch_size= 1, verbose= True)
X_train_adv = attack.generate(x=X_train)
history = model.fit(X_train_adv, Y_train,
                    epochs=2,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model.evaluate(X_test, Y_test, batch_size=14)
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy
#print("%s: %.2f%% :" % (model.metrics_names[2], score[2]*100)) #  cosine

In [ ]:
attack = CarliniLInfMethod(classifier=classifier, confidence= 0.0, targeted= False, learning_rate= 0.01, max_iter= 10
                           , max_halving= 5, max_doubling= 5, eps= 0.3, batch_size= 128, verbose= False)
X_train_adv = attack.generate(x=X_train)
history = model.fit(X_train_adv, Y_train,
                    epochs=2,
                    batch_size=14, validation_split=0.1, verbose=1)
score = model.evaluate(X_test, Y_test, batch_size=14)
print("%s: %.2f%% :" % (model.metrics_names[0], score[0]*100)) #Loss
print("%s: %.2f%% :" % (model.metrics_names[1], score[1]*100)) #Accuracy
#print("%s: %.2f%% :" % (model.metrics_names[2], score[2]*100)) #  cosine